<h1 align="center">Analítica de datos para la toma de decisiones empresariales</h1>
<h1 align="center">Ejemplo: Identificación de datos raros (outliers)</h1>
<h1 align="center">Centro de Educación Continua</h1>
<h1 align="center">EAFIT</h1>
<h1 align="center">2023</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/CFD_Applied/blob/master/figs/CC-BY.png?raw=true">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

## Introducción

La detección de valores atípicos, también conocida como detección de anomalías, es una tarea esencial en la ciencia de datos y se enfoca en identificar puntos de datos con valores extremos en comparación con la distribución general. Esta técnica encuentra diversas aplicaciones en la toma de decisiones empresariales, abarcando desde la detección de fraudes y ciberataques hasta la identificación de falsificaciones de billetes en el sector financiero y médico. En cada uno de estos escenarios, los valores atípicos señalan eventos raros o inusuales que pueden impactar significativamente en la estrategia de la empresa.

Python ofrece una variedad de métodos y paquetes para la detección de valores atípicos. Al seleccionar un método, es crucial considerar la modalidad de los datos, es decir, el número de picos en su distribución. Además, la dimensionalidad y el tamaño del conjunto de datos también juegan un papel importante en la elección del método adecuado. Desde métodos estadísticos simples hasta enfoques más avanzados como los bosques de aislamiento y OneClassSVM, la selección del método depende de la naturaleza y el volumen de los datos.

En el siguiente ejemplo, se exploran distintos métodos para identificar valores atípicos en el conjunto de datos de detección de billetes falsos de un banco suizo. Este caso demuestra cómo la detección de valores atípicos puede ofrecer información valiosa para abordar desafíos específicos en el contexto de la analítica de datos y la toma de decisiones empresariales.

## Carga de datos y análisis exploratorio

In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
df = pd.read_csv("Data/banknotes.csv")
df

In [ ]:
df.info()

## Using Box Plots for Outlier Detection 

In [ ]:
sns.boxplot(data=df,x=df["Length"])
plt.title("Boxplot of Swiss Banknote Length ")

Los puntos en los diagramas de caja corresponden a valores atípicos extremos. Podemos validar que estos son valores atípicos filtrando nuestro DataFrame y usando el método `count` para contar el número de falsificaciones:

In [ ]:
df_outlier1 = df[df['Length']> 216].copy()
print(Counter(df_outlier1['conterfeit']))

Vemos que, con estas condiciones, sólo capturamos uno de cada 100 billetes falsos. Si relajamos las condiciones de filtrado para capturar valores atípicos adicionales, veremos que también capturamos billetes auténticos:

In [ ]:
df_outlier2 = df[df['Length']> 215.5].copy()
print(Counter(df_outlier2['conterfeit']))

Esto corresponde a una precisión de 0,30, lo que no es un gran rendimiento. Peor aún, esto corresponde a una precisión del 1,5 por ciento.

Para ayudar a abordar esta inexactitud, podemos buscar columnas adicionales en los diagramas de caja. Creemos diagramas de caja para las columnas restantes y una función que nos permita generar diagramas de caja para cualquier columna numérica:

In [ ]:
def boxplot(column):
    plt.subplot(3, 2, i+1)  # Define la posición del gráfico en la matriz 3x2
    sns.boxplot(data=df, x=df[f"{column}"])
    plt.title(f"Boxplot of Swiss Banknote {column}")

plt.figure(figsize=(6, 9))  # Tamaño total de la figura
columns = ['Length', 'Right', 'Left', 'Bottom', 'Top', 'Diagonal']

for i, column in enumerate(columns):
    boxplot(column)

plt.tight_layout()  # Ajusta la disposición de los gráficos
plt.show()

Y llamemos a la función con las columnas de longitud, izquierda, derecha, abajo, arriba y diagonal:

Podemos filtrar por el 50 por ciento superior por longitud, derecha, izquierda e inferior:

In [ ]:
df_outlier3 = df[(df['Length']> 215)&(df['Right']> 130)&(df['Left']> 
130)&(df['Bottom']> 10)].copy()
print(Counter(df_outlier3['conterfeit']))

Vemos que ahora capturamos ocho falsificaciones. Aunque esto es una mejora con respecto al único billete falso que capturamos antes, todavía omitimos 92 falsificaciones adicionales, lo que corresponde a una precisión del cuatro por ciento. Además, cuantas más columnas numéricas tengamos en nuestros datos, más engorrosa se vuelve la tarea de detección de valores atípicos. Por esta razón, los diagramas de caja son ideales para conjuntos de datos pequeños y simples con pocas columnas.

## Uso de "Isolation Forest" para la detección de valores atípicos

Isolation Forest es un método de detección de valores atípicos que funciona seleccionando columnas aleatoriamente y sus valores para separar diferentes partes de los datos. Funciona bien con datos más complejos, como conjuntos con muchas más columnas y valores numéricos multimodales.

Importemos el paquete IsolationForest y ajustémoslo a las columnas de longitud, izquierda, derecha, inferior, superior y diagonal. Tenga en cuenta que este algoritmo solo recibe entradas porque es una técnica de aprendizaje automático no supervisada, a diferencia de las técnicas de aprendizaje automático supervisadas, que se entrenan tanto en funciones como en objetivos. Afortunadamente, aún podemos validar nuestras predicciones porque nuestros datos vienen con etiquetas falsificadas.

Primero, importemos los paquetes necesarios:

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
import numpy as np

A continuación, definamos nuestra entrada y salida (solo usaremos esto para validación, no para entrenamiento) y dividamos nuestros datos:

In [ ]:
X = df[['Length', 'Left', 'Right', 'Bottom', 'Top', 'Diagonal']]
y = df['conterfeit']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
test_size=0.33, random_state=42)

A continuación, ajustemos nuestro modelo a nuestras entradas:

In [ ]:
clf = IsolationForest(random_state=0)
clf.fit(X_train)
y_pred = clf.predict(X_test)

Finalmente, predigamos los datos de la prueba y evaluemos la puntuación de precisión. Para nuestros propósitos aquí, validaremos para tener una idea de qué tan bien los métodos pueden detectar valores atípicos:

In [ ]:
pred = pd.DataFrame({'pred': y_pred})
pred['y_pred'] = np.where(pred['pred'] == -1, 1, 0)
y_pred = pred['y_pred'] 
print("Precision:", precision_score(y_test, y_pred))

Vemos que nuestro modelo de detección de valores atípicos tiene una precisión de 0,625. Compare esto con la precisión de 0,30 que logramos con los diagramas de caja. Este modelo también ofrece una precisión del 56 por ciento, en comparación con el cuatro por ciento de los diagramas de caja, lo que muestra una mejora significativa en la detección de valores atípicos. Esto se debe a que los *Isolation Forest* pueden dividir los datos e identificar valores atípicos en múltiples características.

Cuando utilizamos diagramas de caja, tenemos que inspeccionar manualmente los valores atípicos e intentar sacar conclusiones utilizando múltiples características, lo que se vuelve cada vez más difícil cuanto mayor es el número de características. Por ejemplo, puede tener un grupo de puntos donde los valores de características individuales pueden no ser valores atípicos, pero una combinación de valores puede ser anómala. Este tipo de comportamiento es difícil de detectar mediante la inspección de diagramas de caja.